In [1]:
import pandas as pd
import numpy as np
import os
import warnings
import datetime as dt

# Importing sector DFs

In [40]:
#warnings.simplefilter(action='ignore', category='DtypeWarning')

PATH_TO_DATA = '/Users/R/code/HaukeFock/predict_energy_generation/raw_data/energy_generation/'
companies = !ls /Users/R/code/HaukeFock/predict_energy_generation/raw_data/energy_generation/
columns = ['Date', 'Time of day', 'Biomass[MWh]', 'Hydropower[MWh]',
       'Wind offshore[MWh]', 'Wind onshore[MWh]', 'Photovoltaics[MWh]',
       'Other renewable[MWh]', 'Nuclear[MWh]', 'Fossil brown coal[MWh]',
       'Fossil hard coal[MWh]', 'Fossil gas[MWh]', 'Hydro pumped storage[MWh]',
       'Other conventional[MWh]']
dataframes = {}

for company in companies:
    #print(company)
    exec(f"df_{company} = pd.DataFrame(columns=columns)")
    data_paths = !ls /Users/R/code/HaukeFock/predict_energy_generation/raw_data/energy_generation/{company}
    for file in data_paths:
        #print(file)
        data = pd.read_csv(f'{PATH_TO_DATA}/{company}/{file}', sep=';')
        exec(f"df_{company} = df_{company}.append(data, ignore_index=True)")
    exec(f"dataframes['df_{company}'] = df_{company}")
    print(f'df_{company} Loaded!!!')

/Users/R/.pyenv/versions/3.8.6/envs/predict_energy_generation/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3169: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
/Users/R/.pyenv/versions/3.8.6/envs/predict_energy_generation/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3169: DtypeWarning: Columns (4,11) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


df_50hertz Loaded!!!


/Users/R/.pyenv/versions/3.8.6/envs/predict_energy_generation/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3169: DtypeWarning: Columns (2,3,4,6,9,10,12) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
/Users/R/.pyenv/versions/3.8.6/envs/predict_energy_generation/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3169: DtypeWarning: Columns (2,3,5,6) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
/Users/R/.pyenv/versions/3.8.6/envs/predict_energy_generation/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3169: DtypeWarning: Columns (5,12) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


df_amprion Loaded!!!
df_tennet Loaded!!!


/Users/R/.pyenv/versions/3.8.6/envs/predict_energy_generation/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3169: DtypeWarning: Columns (11) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
/Users/R/.pyenv/versions/3.8.6/envs/predict_energy_generation/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3169: DtypeWarning: Columns (5,11) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


df_transnet_bw Loaded!!!


/Users/R/.pyenv/versions/3.8.6/envs/predict_energy_generation/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3169: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [41]:
df_tennet['Datum'][:70272]

0        Jan 1, 2015
1        Jan 1, 2015
2        Jan 1, 2015
3        Jan 1, 2015
4        Jan 1, 2015
            ...     
70267    Jan 1, 2017
70268    Jan 1, 2017
70269    Jan 1, 2017
70270    Jan 1, 2017
70271    Jan 1, 2017
Name: Datum, Length: 70272, dtype: object

In [42]:
# dropping unecessary columns 
df_50hertz['Date'] = df_50hertz['Date'][:140448].append(df_50hertz['Datum'][140448:])
df_50hertz['Time of day'] = df_50hertz['Time of day'][:140448].append(df_50hertz['Uhrzeit'][140448:])

df_tennet['Date'] = df_tennet['Datum'][:70272].append(df_tennet['Date'][70272:])
df_tennet['Time of day'] = df_tennet['Uhrzeit'][:70272].append(df_tennet['Time of day'][70272:])

df_50hertz.drop(columns=['Datum', 'Uhrzeit'], inplace=True)
df_tennet.drop(columns=['Datum', 'Uhrzeit'], inplace=True)

# missing values resume for each area
for key, df in dataframes.items():
    if (df.columns != columns).all():
        print(f'{key} has different columns than expected!')
    else:
        print(f'>>>>>>>>>>>>>> {key} <<<<<<<<<<<<<<\n')
        print(f'Shape: {df.shape}\n')
        print(f'NaN values: \n{df.isna().sum()}\n')

>>>>>>>>>>>>>> df_50hertz <<<<<<<<<<<<<<

Shape: (210720, 14)

NaN values: 
Date                              0
Time of day                       0
Biomass[MWh]                      0
Hydropower[MWh]                   0
Wind offshore[MWh]                0
Wind onshore[MWh]                 0
Photovoltaics[MWh]                0
Other renewable[MWh]              0
Nuclear[MWh]                 210720
Fossil brown coal[MWh]            0
Fossil hard coal[MWh]             0
Fossil gas[MWh]                   0
Hydro pumped storage[MWh]         0
Other conventional[MWh]           0
dtype: int64

>>>>>>>>>>>>>> df_amprion <<<<<<<<<<<<<<

Shape: (210720, 14)

NaN values: 
Date                              0
Time of day                       0
Biomass[MWh]                      0
Hydropower[MWh]                   0
Wind offshore[MWh]           210720
Wind onshore[MWh]                 0
Photovoltaics[MWh]                0
Other renewable[MWh]              0
Nuclear[MWh]                      0
Fossil

In [44]:
numeric_values = ['Biomass[MWh]',
                 'Hydropower[MWh]',
                 'Wind offshore[MWh]',
                 'Wind onshore[MWh]',
                 'Photovoltaics[MWh]',
                 'Other renewable[MWh]',
                 'Nuclear[MWh]',
                 'Fossil brown coal[MWh]',
                 'Fossil hard coal[MWh]',
                 'Fossil gas[MWh]',
                 'Hydro pumped storage[MWh]',
                 'Other conventional[MWh]']

# set the features to numerical values
for column in numeric_values:
    #print(column)
    for key, df in dataframes.items():
        #print(f'{key} => {column}')
        df[column] = df[column].apply(lambda x: str(x).replace(',',''))
        df[column] = pd.to_numeric(df[column], errors='coerce')

In [45]:
# join 'Date' and 'Time of Day' into one
for df in dataframes.keys():
    print(f'Working on {df}')
    exec(f"{df}['Date'] = pd.to_datetime({df}['Date']).apply(lambda x: x.strftime('%Y-%m-%d '))")
    exec(f"{df}['Time of day'] = pd.to_datetime({df}['Time of day']).apply(lambda x: x.strftime('%H:%M'))")
    exec(f"{df}['Date'] = pd.to_datetime({df}['Date'] + {df}['Time of day'])")
    exec(f"{df}.drop(columns=['Time of day'], inplace=True)")

Working on df_50hertz
Working on df_amprion
Working on df_tennet
Working on df_transnet_bw


# Group by daily energy generation

In [46]:
# group energy generation per day 
for df in dataframes.keys():
    exec(f"{df} = {df}.groupby(by={df}['Date'].dt.date).mean().reset_index()")

# Creating complete DF

In [47]:
df_deutschland = pd.DataFrame()

for df, company in zip(dataframes.keys(), companies):
        exec(f"{df}['region'] = company")
        exec(f"df_deutschland = df_deutschland.append({df}, ignore_index=True)")

df_deutschland.shape

(8772, 14)

In [48]:
df_deutschland.head()

,Date,Biomass[MWh],Hydropower[MWh],Wind offshore[MWh],Wind onshore[MWh],Photovoltaics[MWh],Other renewable[MWh],Nuclear[MWh],Fossil brown coal[MWh],Fossil hard coal[MWh],Fossil gas[MWh],Hydro pumped storage[MWh],Other conventional[MWh],region
0,2015-01-01,305.739583,35.916667,11.989583,1278.427083,75.531250,7.020833,NaN,1485.166667,107.614583,104.593750,135.583333,134.875000,50hertz
1,2015-01-02,301.916667,35.427083,9.947917,2555.510417,17.677083,15.031250,NaN,1011.666667,58.250000,102.864583,128.708333,144.135417,50hertz
2,2015-01-03,301.875000,35.510417,11.281250,2308.958333,27.354167,18.718750,NaN,1146.885417,70.250000,123.062500,102.281250,147.968750,50hertz
3,2015-01-04,301.208333,35.406250,11.354167,2079.041667,49.750000,18.479167,NaN,1156.572917,81.427083,107.166667,121.385417,148.572917,50hertz
4,2015-01-05,298.187500,35.020833,4.104167,1333.343750,20.187500,18.093750,NaN,1742.645833,250.822917,159.322917,104.562500,146.375000,50hertz
